# iterate over folder with images, count images with the same size

In [19]:
## Produce dictionary with key:count, where key is size of image

import cv2, os
from tqdm.notebook import tqdm
path = "data/DIV2K_train_HR"
sizes={}
for file in tqdm(os.listdir("data/DIV2K_train_HR")):
    image = cv2.imread(path + '/' + file)
    if image.shape in sizes.keys():
        sizes[image.shape]+=1
    else:
        sizes[image.shape]=1

(1404, 2040, 3)

In [ ]:
dict(sorted(sizes.items(), key=lambda item: item[0]))

#### Results
There is 429 images of size (1356, 2040, 3)

## Read all images with selected size and put them into np.array
Getting train high resolution


In [33]:
# Getting train high resolution
import cv2, os
from tqdm.notebook import tqdm
import numpy as np
from IPython.display import clear_output, display
path = "data/DIV2K_train_HR"
img_HQ={} # a list of np.ndarrays() os shape (1356, 2040, 3)
for file in tqdm(os.listdir(path)):
    image = cv2.imread(path + '/' + file)
    if image.shape ==  (1356, 2040, 3):
        index=file.split('.')[0]
        img_HQ[index]= image

  0%|          | 0/800 [00:00<?, ?it/s]

In [39]:
# Getting train low resolution x8, corresponding to gotten High Resolution
import cv2, os
from tqdm.notebook import tqdm
import numpy as np
from IPython.display import clear_output, display
path = "data/DIV2K_train_LR_x8"
img_LQ={}

for file in tqdm(os.listdir(path)):
    image = cv2.imread(path + '/' + file)
    index=file.split('x')[0]
    if index in img_HQ.keys():
        img_LQ[index]= image

  0%|          | 0/800 [00:00<?, ?it/s]

In [42]:
print(f"Those two should match:  {len(img_HQ)=}, {len(img_LQ)=}")

Those two should match:  len(img_HQ)=429, len(img_LQ)=429


### old code

In [ ]:
# pip install fiftyone
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    classes=["cat", "dog"],
    max_samples=25,
)

session = fo.launch_app(dataset)

In [14]:
import math
from torch import nn

class ESPCN(nn.Module):
    def __init__(self, scale_factor, num_channels=1):
        super(ESPCN, self).__init__()
        self.first_part = nn.Sequential(
            nn.Conv2d(num_channels, 64, kernel_size=5, padding=5//2),
            nn.Tanh(),
            nn.Conv2d(64, 32, kernel_size=3, padding=3//2),
            nn.Tanh(),
        )
        self.last_part = nn.Sequential(
            nn.Conv2d(32, num_channels * (scale_factor ** 2), kernel_size=3, padding=3 // 2),
            nn.PixelShuffle(scale_factor)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m.in_channels == 32:
                    nn.init.normal_(m.weight.data, mean=0.0, std=0.001)
                    nn.init.zeros_(m.bias.data)
                else:
                    nn.init.normal_(m.weight.data, mean=0.0, std=math.sqrt(2/(m.out_channels*m.weight.data[0][0].numel())))
                    nn.init.zeros_(m.bias.data)

    def forward(self, x):
        x = self.first_part(x)
        x = self.last_part(x)
        return x